推荐算法
- 推荐模型构建流程
- 推荐算法概述
- 基于协同过滤算法
- 协同过滤实现

推荐算法构建流程
- 数据采集
- 数据清洗
- 特征工程
- ML算法
- 推理
- 评估

1. 数据采集
- 显性数据
    - 打分
    - 评论
- 隐性数据
    - 历史订单
    - 购物车
    - 浏览记录
    - 点击
    - 搜索记录

2. 特征工程
- 协同过滤： User-Item Matrix
- 基于内容： 分词、embedding

3. 训练模型
- 协同过滤
    - KNN
    - 矩阵分解
4. 评估、上线

 

经典推荐算法： 协同过滤 Collaborative Filtering

基于以下假设：
- 跟你喜好相似的人喜欢的东西，你也可能喜欢 （User-based CF）
- 跟你喜欢的东西相似的东西，你也可能喜欢   （Item-based CF）

实现协同过滤步骤：
1. 找出最相似的人或物品
2. 根据最相似的人或物品产生推荐结果 （初始推荐结果）
3. 过滤掉原本就为某用户推荐的物品

- User-based CF
    - 给用户A找到最相似的N个用户
    - N个用户消费过哪些产品
    - N个用户消费过的产品 - A用户消费过的就是推荐结果

- Item-based CF
    - 给产品1找到组相似的N个产品
    - 某用户消费过的产品中将每个产品都扩展为最相似的N个产品
    - 过滤、去重就是推荐结果

相似度计算
- 相似度计算
    - 实数值问题
    - 布尔值问题

- 欧氏距离
- 余弦相似度
$$ cos\theta = \frac{\textbf{ab}}{|\textbf{a}| |\textbf{b}|}$$

- 皮尔逊相关系数 Pearson's Correlation
    - 特征向量去中心化版本的余弦相似度
    - 皮尔逊相关系数衡量两个变量$X$和$Y$之间的线性相关性，其定义为：

$$ \rho_{X,Y} = \frac{\text{cov}(X,Y)}{\sigma_X \sigma_Y} = \frac{\sum_{i=1}^n (x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum_{i=1}^n (x_i - \bar{x})^2} \sqrt{\sum_{i=1}^n (y_i - \bar{y})^2}} $$


- 杰卡德相似度 Jaccard coefficient
    - 交并比
    - 适合处理布尔值问题

In [14]:
import pandas as pd

users = ["User1", "User2", "User3", "User4", "User5"]
items = ["ItemA", "ItemB", "ItemC", "ItemD", "ItemE"]

# 用户购买记录
dataset = [
    [1, 0, 1, 1, 0],
    [1, 0, 0, 1, 1],
    [1, 0, 1, 0, 0],
    [0, 1, 0, 1, 1],
    [1, 1, 1, 0, 1]
]

df = pd.DataFrame(dataset, columns=items, index=users)
X = df.values
print(df)

       ItemA  ItemB  ItemC  ItemD  ItemE
User1      1      0      1      1      0
User2      1      0      0      1      1
User3      1      0      1      0      0
User4      0      1      0      1      1
User5      1      1      1      0      1


In [17]:
from sklearn.metrics import jaccard_score

jaccard_score(df["ItemA"], df["ItemB"])

np.float64(0.2)

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances

user_similiarity = 1 - pairwise_distances(X, metric='jaccard')

user_similiarity = pd.DataFrame(user_similiarity, columns=users, index=users)

/opt/anaconda3/envs/recommender/lib/python3.12/site-packages/sklearn/metrics/pairwise.py:2466: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [20]:
user_similiarity

,User1,User2,User3,User4,User5
User1,1.000000,0.50,0.666667,0.2,0.4
User2,0.500000,1.00,0.250000,0.5,0.4
User3,0.666667,0.25,1.000000,0.0,0.5
User4,0.200000,0.50,0.000000,1.0,0.4
User5,0.400000,0.40,0.500000,0.4,1.0


In [23]:
item_similarity = 1 - pairwise_distances(X.T, metric='jaccard')
item_similarity = pd.DataFrame(item_similarity, columns=items, index=items)

/opt/anaconda3/envs/recommender/lib/python3.12/site-packages/sklearn/metrics/pairwise.py:2466: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [24]:
item_similarity

,ItemA,ItemB,ItemC,ItemD,ItemE
ItemA,1.00,0.200000,0.75,0.40,0.400000
ItemB,0.20,1.000000,0.25,0.25,0.666667
ItemC,0.75,0.250000,1.00,0.20,0.200000
ItemD,0.40,0.250000,0.20,1.00,0.500000
ItemE,0.40,0.666667,0.20,0.50,1.000000


In [45]:
topN_users = {}

for i in user_similiarity.index:
    # 取出每一列数据， 删除自己
    _df = user_similiarity.loc[i].drop([i])

    # 排序
    _df_sorted = _df.sort_values(ascending=False)

    #挑出最相似的两个
    top2 = list(_df_sorted.index[:2])
    topN_users[i] = top2

In [46]:
topN_users

{'User1': ['User3', 'User2'],
 'User2': ['User1', 'User4'],
 'User3': ['User1', 'User5'],
 'User4': ['User2', 'User5'],
 'User5': ['User3', 'User1']}

In [47]:
import numpy as np

# 构建推荐结果
rs_results = {}

for user, sim_users in topN_users.items():
    rs_result = set() # 为每一个用户保存推荐结果
    for sim_user in sim_users:
        rs_result = rs_result.union(set(df.loc[sim_user].replace(0, np.nan).dropna().index))

    # 过滤已经购买的商品
    rs_result -= set(df.loc[user].replace(0, np.nan).dropna().index)
    rs_results[user] = rs_result

In [43]:
rs_results

{'User1': {'ItemE'},
 'User2': {'ItemB', 'ItemC'},
 'User3': {'ItemB', 'ItemD', 'ItemE'},
 'User4': {'ItemA', 'ItemC'},
 'User5': {'ItemD'}}

协同过滤是一个非常直观、可解释性很强的模型,但它并不具备 较强的泛化能力,换句话说,协同过滤无法将两个物品相似这一信息 推广到其他物品的相似性计算上。这就导致了一个比较严重的问题 ——热门的物品具有很强的头部效应,容易跟大量物品产生相似性; 而尾部的物品由于特征向量稀疏,很少与其他物品产生相似性,导致 很少被推荐。